Note that most of this notebook directly come from online source, the code layout is pretty bad, might take a while to understand what the original author is writing 

In [1]:
import csv                               # csv reader
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
from random import shuffle
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\edton\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import random
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [3]:
from utils.loader import DataLoader
loader = DataLoader()
loader.testing()
gg = loader.load_gold_data(_type = 'dec', neg_polarity=True, pos_polarity=True)

updated2


In [4]:
_type = 'dedc'
if _type not in DataLoader().types.keys():
    print('np')

np


In [66]:
# load data from a file and append it to the rawData
def loadData(shuffle = True):
    dec_data = loader.load_gold_data(_type = 'dec', neg_polarity=True, pos_polarity=True)
    truth_data = loader.load_gold_data(_type = 'truth', neg_polarity=True, pos_polarity=True)

    for i in range(len(dec_data)):
        rawData.append((dec_data[i], 'fake'))
        rawData.append((truth_data[i], 'real'))

    if shuffle:
        random.shuffle(rawData)

def sentiment_split(data):
    strong = []
    weak = []
    analyser = SentimentIntensityAnalyzer()

    for i in range(len(data)):
        vadersenti = analyser.polarity_scores(data[i][0])
        if abs(vadersenti['compound']) > 0.7:
            strong.append(data[i])
        else:
            weak.append(data[i])
    
    return strong, weak
        

def splitData(percentage, data):
    dataSamples = len(data)
    halfOfData = int(len(data)/2)
    trainingSamples = int((percentage*dataSamples)/2)
    for (Text, Label) in data[:trainingSamples] + data[halfOfData:halfOfData+trainingSamples]:
        trainData.append((toFeatureVector(preProcess(Text)),Label))
        # trainData.append((toFeatureVector(Rating, verified_Purchase, product_Category, preProcess(Text)),Label))
    for (Text, Label) in data[trainingSamples:halfOfData] + data[halfOfData+trainingSamples:]:
        testData.append((toFeatureVector(preProcess(Text)),Label))
# QUESTION 1


# Convert line from input file into an id/text/label tuple
def parseReview(reviewLine):
    # Should return a triple of an integer, a string containing the review, and a string indicating the label
    s=""
    if reviewLine[1]=="__label1__":
        s = "fake"
    else: 
        s = "real"
    return (reviewLine[8], s)
# TEXT PREPROCESSING AND FEATURE VECTORIZATION
# Input: a string of one review
table = str.maketrans({key: None for key in string.punctuation})


def preProcess(text):
    # Should return a list of tokens
    lemmatizer = WordNetLemmatizer()
    filtered_tokens=[]
    lemmatized_tokens = []
    stop_words = set(stopwords.words('english'))
    text = text.translate(table)
    for w in text.split(" "):

        ######################## try without removing ################################################################
        lemmatized_tokens.append(lemmatizer.lemmatize(w.lower()))
        filtered_tokens = [' '.join(l) for l in nltk.bigrams(lemmatized_tokens)] + lemmatized_tokens

        ######################## try keep stopwords only ################################################################
        # if w in stop_words:
        #     lemmatized_tokens.append(lemmatizer.lemmatize(w.lower()))
        # filtered_tokens = [' '.join(l) for l in nltk.ngrams(lemmatized_tokens, n=1)] + lemmatized_tokens

        ######################## try removing ################################################################
        # if w not in stop_words:
        #     lemmatized_tokens.append(lemmatizer.lemmatize(w.lower()))
        # filtered_tokens = [' '.join(l) for l in nltk.bigrams(lemmatized_tokens)] + lemmatized_tokens
    return filtered_tokens
featureDict = {} # A global dictionary of features


def toFeatureVector(tokens):
    localDict = {}
            
#Text        

    for token in tokens:
        if token not in featureDict:
            featureDict[token] = 1
        else:
            featureDict[token] = +1
            
        if token not in localDict:
            localDict[token] = 1
        else:
            localDict[token] = +1
    
    return localDict

In [6]:
import os
from utils.configs import config
from utils.loader import DataLoader



In [7]:
# analyser = SentimentIntensityAnalyzer()

# for i in range(100):
#     vadersenti = analyser.polarity_scores(rawData[i][0])
#     print(vadersenti)

In [8]:
amazon_data = loader.load_amazon(deceptive=False, all=True, test_mode=True)

In [11]:
rawAmazon = []
for i in range(len(amazon_data)):
    if amazon_data['LABEL'][i] == 1:
        rawAmazon.append((amazon_data['REVIEW_TEXT'][i], 'fake'))
    else:
        rawAmazon.append((amazon_data['REVIEW_TEXT'][i], 'real'))

In [12]:
rawAmazon[0]

('I a so happy I bought this. I loved the show but I missed some of them. Thank you for putting it on sale. I hope others would watch the show and get hooked like I did.',
 'real')

In [13]:
testAmazon = []
for (Text, Label) in rawAmazon:
    testAmazon.append((toFeatureVector(preProcess(Text)),Label))

In [67]:
# MAIN

# loading reviews
rawData = []          # the filtered data from the dataset file (should be 21000 samples)
preprocessedData = [] # the preprocessed reviews (just to see how your preprocessing is doing)
trainData = []        # the training data as a percentage of the total dataset (currently 80%, or 16800 samples)
testData = []         # the test data as a percentage of the total dataset (currently 20%, or 4200 samples)

# the output classes
fakeLabel = 'fake'
realLabel = 'real'

# references to the data files
path = os.path.join(config['base_path'], config['amazon_path'], 'train')
reviewPath = DataLoader.list_all_txt_files(path)[0]

## Do the actual stuff
# We parse the dataset and put it in a raw data list
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing the dataset...",sep='\n')

loadData(shuffle = True)
strong, weak = sentiment_split(rawData)

# We split the raw dataset into a set of training data and a set of test data (80/20)
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Preparing training and test data...",sep='\n')


Now 0 rawData, 0 trainData, 0 testData
Preparing the dataset...
Now 1600 rawData, 0 trainData, 0 testData
Preparing training and test data...


In [68]:
for i in range(5):
    print(weak[i][1])

fake
real
real
fake
real


In [69]:
trainData = []        # the training data as a percentage of the total dataset (currently 80%, or 16800 samples)
testData = []         # the test data as a percentage of the total dataset (currently 20%, or 4200 samples)

splitData(0.8, weak)
# We print the number of training samples and the number of features
print("Now %d rawData, %d trainData, %d testData" % (len(rawData), len(trainData), len(testData)),
      "Training Samples: ", len(trainData), "Features: ", len(featureDict), sep='\n')

Now 1600 rawData, 250 trainData, 64 testData
Training Samples: 
250
Features: 
28180


In [32]:
# TRAINING AND VALIDATING OUR CLASSIFIER
def trainClassifier(trainData):
    print("Training Classifier...")
    pipeline =  Pipeline([('svc', LinearSVC())])
    return SklearnClassifier(pipeline).train(trainData)


def predictLabels(reviewSamples, classifier):
    return classifier.classify_many(map(lambda t: t[0], reviewSamples))
    

def crossValidate(dataset, folds):
    shuffle(dataset)
    cv_results = []
    foldSize = int(len(dataset)/folds)
    for i in range(0,len(dataset),foldSize):
        classifier = trainClassifier(dataset[:i]+dataset[foldSize+i:])
        y_pred = predictLabels(dataset[i:i+foldSize],classifier)
        a = accuracy_score(list(map(lambda d : d[1], dataset[i:i+foldSize])), y_pred)
        (p,r,f,_) = precision_recall_fscore_support(list(map(lambda d : d[1], dataset[i:i+foldSize])), y_pred, average ='macro')
        #print(a,p,r,f)
        cv_results.append((a,p,r,f))
    cv_results = (np.mean(np.array(cv_results),axis=0))
    return cv_results

In [33]:
# additional classifiers
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier


def MNB_Classifier(trainData):
    print("Training Classifier...")
    MNB_classifier = SklearnClassifier(MultinomialNB())
    return MNB_classifier.train(trainData)


def BernoulliNB_Classifier(trainData):
    print("Training Classifier...")
    BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
    return BernoulliNB_classifier.train(trainData)


def LogisticRegression_Classifier(trainData):
    print("Training Classifier...")
    LogisticRegression_classifier = SklearnClassifier(LogisticRegression(max_iter=1000))
    return LogisticRegression_classifier.train(trainData)

def SGDClassifier_Classifier(trainData):
    print("Training Classifier...")
    SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
    return SGDClassifier_classifier.train(trainData)



In [34]:
len(testAmazon)

5250

In [35]:
len(testData)

320

In [83]:
####################################################
# run this to change the testData

# test_Data = testAmazon
test_Data = testData


####################################################


In [84]:
#  TEST DATA
classifier = trainClassifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
accuracy:  0.859375
Precision:  0.8607038123167156
Recall:  0.859375
f1-score:  0.8593406593406594


In [85]:
#  TEST DATA
classifier = MNB_Classifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
accuracy:  0.890625
Precision:  0.8911330049261084
Recall:  0.890625
f1-score:  0.8899533284205355


In [86]:
#  TEST DATA
classifier = BernoulliNB_Classifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
accuracy:  0.5625
Precision:  0.7586206896551724
Recall:  0.5625
f1-score:  0.49090909090909096


In [87]:
#  TEST DATA
classifier = LogisticRegression_Classifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
accuracy:  0.859375
Precision:  0.8607038123167156
Recall:  0.859375
f1-score:  0.8593406593406594


In [88]:
#  TEST DATA
classifier = SGDClassifier_Classifier(trainData)
predictions = predictLabels(test_Data, classifier)
true_labels = list(map(lambda d: d[1], test_Data))
a = accuracy_score(true_labels, predictions)
p, r, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='macro')
print("accuracy: ", a)
print("Precision: ", p)
print("Recall: ", a)
print("f1-score: ", f1)

Training Classifier...
accuracy:  0.859375
Precision:  0.8596059113300492
Recall:  0.859375
f1-score:  0.8585114222549741


In [82]:
test_samples = testData
pred = predictLabels(test_samples, classifier)
summ = 0
for i in range(len(test_samples)):
    correct = np.where(test_samples[i][1] == pred[i], 1, 0)
    # print(f'True: {test_samples[i][1]}, Predict: {pred[i]},   {correct}')
    summ += correct

print(f'Acc: {summ/len(test_samples)}')


Acc: 0.84375


In [26]:
type(trainData)

list

In [68]:
print(trainData[1][0], trainData[1][1])

{'wa': 1, 'with': 1, 'but': 1, 'a': 1, 'to': 1, 'while': 1, 'my': 1, 'there': 1, 'for': 1, 'few': 1, 'and': 1, 'we': 1, 'the': 1, 'on': 1, 'their': 1, 'did': 1, 'is': 1, 'all': 1, 'of': 1, 'have': 1, 'at': 1, 'will': 1} fake


In [28]:
type(trainData[1][0])

dict